<a href="https://colab.research.google.com/github/sankarakokin/DataScience_1082/blob/master/Copia_de_nifty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks base::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [5]:

library(tidyverse)
library(rvest)
rm(list=ls())
library(curl)
library(rvest)


url <- "https://query1.finance.yahoo.com/v7/finance/download/0P00006XPH.F?period1=1646726400&period2=1716793908&interval=1d&events=history&includeAdjustedClose=true"
#url<-"https://query1.finance.yahoo.com/v7/finance/download/%5EIBEX?period1=742460400&period2=1716779030&interval=1d&events=history&includeAdjustedClose=true"
urlinr<-"https://query1.finance.yahoo.com/v7/finance/download/EURINR%3DX?period1=1070236800&period2=1716791436&interval=1d&events=history&includeAdjustedClose=true"
#url<-"https://query1.finance.yahoo.com/v7/finance/download/0P0000PU5O.F?period1=1685092055&period2=1716714455&interval=1d&events=history&includeAdjustedClose=true"
filename <- "NSEI_historical_data.csv"
filename1 <- "eurinr.csv"
#NSEI IBEX
#file.remove("C:\Users\a\Documents\Inteligencia_Artificial\NSEI_historical_data.csv")
#C:\Users\a\Documents\Inteligencia_Artificial
file.remove("NSEI_historical_data.csv")
file.remove("eurinr.csv")
download.file(url, filename, mode = "wb")
download.file(urlinr, filename1, mode = "wb")
# View the extracted table

# Load the data

nifty50_data <- read.csv("NSEI_historical_data.csv")
eurinr<-read.csv("eurinr.csv")[,c(1,5)]
eurinr$Close=as.numeric(eurinr$Close)
eurinr$Close_inr=1/eurinr$Close
eurinr$Close=NULL
# Convertir columnas de nifty50_df a numÃ©ricas excepto Date
nifty50_data=nifty50_data[nifty50_data$Open!="null",]

nifty50_data[, setdiff(names(nifty50_data), "Date")] <-
  apply(nifty50_data[, setdiff(names(nifty50_data), "Date")], 2, as.numeric)
# Unir df1 y df2 por la variable "date"
eurinr$Date <- as.Date(eurinr$Date, format = "%Y-%m-%d")
nifty50_data$Date <- as.Date(nifty50_data$Date, format = "%Y-%m-%d")
nifty50_data=inner_join(nifty50_data, eurinr, by = "Date")
#nifty50_data=nifty50_data[-c(251:256),]
#nifty50_data$Close=nifty50_data$Close*nifty50_data$Close_inr
z1=sample(75:dim(nifty50_data)[1],1)
z=nifty50_data[z1,]
z2=nifty50_data[z1,]

nifty50_data=nifty50_data[nifty50_data$Date<z2$Date,]
# nifty50_data=nifty50_data[-z1,]

library(dplyr)

# Supongamos que tienes un dataset llamado "mi_dataset" y quieres ordenarlo por la variable "x" en orden ascendente
nifty50_data <- nifty50_data %>% arrange(Date)
nifty50_data=na.omit(nifty50_data)
nifty50_data$glm <- ifelse(as.numeric(gsub(",","",nifty50_data$Close)) < as.numeric(gsub(",","",(lead(nifty50_data$Close,4) ))), 1, 0)

# Create a data frame with the required columns

nifty50_df=nifty50_data

nifty50_df$Close_post = lead(nifty50_data$Close,1)
nifty50_df$glm_post = lead(nifty50_data$glm,1)

# Calculate daily returns
nifty50_df$ret <- c(0, diff(log(nifty50_df$Close)))

# Calculate moving averages
install.packages("TTR")
library(TTR)
nifty50_df$ma_50 <- SMA((nifty50_df$Close), 50)
nifty50_df$ma_100 <- SMA(nifty50_df$Close, 100)

# Calculate relative strength index (RSI)
nifty50_df$rsi <- RSI(lag(nifty50_df$Close,1), 14)

install.packages("randomForest")
library(randomForest)

nifty50_df$Close_post=NULL

x=dim(nifty50_df)[1]
rf_model <- randomForest(glm_post ~ ., data = na.omit(nifty50_df[,-c(1,9)],ntree=1000)  )

summary(rf_model )
#nifty50_df$glm_post=NULL
nifty50_df$Volume <- ifelse(nifty50_df$Volume == 0, lag(nifty50_df$Volume, 1), nifty50_df$Volume)
nifty50_df$rf_pred <- predict(rf_model,  nifty50_df[,])
nifty50_df$glm2 <- ifelse(nifty50_df$rf_pred>0.5,1,0 )

View(nifty50_df)
print(z)

[1] TRUE

[1] TRUE

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”


                Length Class  Mode     
call              3    -none- call     
type              1    -none- character
predicted        33    -none- numeric  
mse             500    -none- numeric  
rsq             500    -none- numeric  
oob.times        33    -none- numeric  
importance       11    -none- numeric  
importanceSD      0    -none- NULL     
localImportance   0    -none- NULL     
proximity         0    -none- NULL     
ntree             1    -none- numeric  
mtry              1    -none- numeric  
forest           11    -none- list     
coefs             0    -none- NULL     
y                33    -none- numeric  
test              0    -none- NULL     
inbag             0    -none- NULL     
terms             3    terms  call     

,Date,Open,High,Low,Close,Adj.Close,Volume,Close_inr,glm,glm_post,ret,ma_50,ma_100,rsi,rf_pred,glm2
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2022-03-08,68.27,68.27,68.27,68.27,68.27,NA,0.01196845,1,1,0.0000000000,NA,NA,NA,NA,NA
2,2022-03-09,69.43,69.43,69.43,69.43,69.43,0,0.01194215,1,1,0.0168486633,NA,NA,NA,NA,NA
3,2022-03-10,70.04,70.04,70.04,70.04,70.04,0,0.01183348,1,1,0.0087474709,NA,NA,NA,NA,NA
4,2022-03-11,70.47,70.47,70.47,70.47,70.47,0,0.01191043,1,1,0.0061205798,NA,NA,NA,NA,NA
5,2022-03-14,71.37,71.37,71.37,71.37,71.37,0,0.01198998,1,1,0.0126905535,NA,NA,NA,NA,NA
6,2022-03-15,70.44,70.44,70.44,70.44,70.44,0,0.01195429,1,1,-0.0131163430,NA,NA,NA,NA,NA
7,2022-03-16,72.13,72.13,72.13,72.13,72.13,0,0.01196516,1,0,0.0237086929,NA,NA,NA,NA,NA
8,2022-03-17,73.49,73.49,73.49,73.49,73.49,0,0.01191782,0,0,0.0186793104,NA,NA,NA,NA,NA
9,2022-03-18,73.15,73.15,73.15,73.15,73.15,0,0.01187606,0,1,-0.0046371605,NA,NA,NA,NA,NA


          Date  Open  High   Low Close Adj.Close Volume  Close_inr
138 2022-09-27 78.49 78.49 78.49 78.49     78.49      0 0.01272815


Rows: 256 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (13): Open, High, Low, Close, Adj.Close, Volume, glm, ret, ma_50, ma_10...
date  (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Date,Open,High,Low,Close,Adj.Close,Volume,glm,ret,ma_50,ma_100,rsi,rf_pred,glm2
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2023-05-25,9169.5,9188.1,9116.1,9116.1,9116.1,123786500,NA,0.0000000000,NA,NA,NA,NA,NA
2023-05-26,9137.8,9210.9,9048.2,9191.1,9191.1,99198600,1,0.0081935438,NA,NA,NA,NA,NA
2023-05-29,9263.1,9271.1,9146.6,9180.1,9180.1,59244300,0,-0.0011975268,NA,NA,NA,NA,NA
2023-05-30,9189.0,9248.3,9156.0,9167.5,9167.5,111494900,0,-0.0013734343,NA,NA,NA,NA,NA
2023-05-31,9096.5,9183.4,9034.9,9050.2,9050.2,401513500,0,-0.0128777425,NA,NA,NA,NA,NA
2023-06-01,9130.3,9181.6,9105.9,9167.5,9167.5,109883600,1,0.0128777425,NA,NA,NA,NA,NA
2023-06-02,9216.2,9324.8,9204.2,9317.3,9317.3,132595800,1,0.0162082453,NA,NA,NA,NA,NA
2023-06-05,9352.0,9382.4,9276.1,9289.1,9289.1,102104900,0,-0.0030312386,NA,NA,NA,NA,NA
2023-06-06,9266.0,9310.8,9242.3,9310.8,9310.8,94770400,1,0.0023333681,NA,NA,NA,NA,NA
